# Simulations on a sheet domain

The following notebook contains sample code for simulating gas exchange on an infinite sheet geometry using the model outlined in Zurita & Hurtado (2022).

First we import necessary libraries and add important directories to the system path.

In [1]:
import sys
import os
import dolfin
import matplotlib.pyplot as plt
plt.style.use('ggplot')  # This is just an aesthetic choice for graphs
import pyvista as pv
sys.path.append(os.getcwd()[:-6])

Following that, we import the main simulation class and parameter dictionary from the model source files.

In [2]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Now we outline the folder name and the path to save raw data files.

In [3]:
folder = "sheet-notebook"
path = os.path.join("../raw-data", folder)

Now we run the solver. First, we instance the model. Then, we generate the mesh that mimics an infinite sheet geometry. Afterwards we prescribe the blood velocity $\boldsymbol{u}$ and simulate the steady transport problem _without_ hemoglobin effects. Finally, we run the actual full model using the previous solution as an initial guess for the Newton iteration, and save the boundary data.

In [4]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.generate_sheet_mesh(
    dims=(300, 6, 6), elems=(100, 5, 5), save=True, periodic=True, refined=True
)
model.set_u(value=(400, 0, 0), save=True)
x = model.sim_t(hb=False, save=False)
solution = model.sim_t(hb=True, save=True, guess=x)
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries

No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 3.817e+07 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-08)
  Newton iteration 1: r (abs) = 3.884e-08 (tol = 1.000e-08) r (rel) = 1.018e-15 (tol = 1.000e-08)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.322e+09 (tol = 1.000e-08) r (rel) = 1.000e+00 (tol = 1.000e-08)
  Newton iteration 1: r (abs) = 6.241e+07 (tol = 1.000e-08) r (rel) = 4.721e-02 (tol = 1.000e-08)
  Newton iteration 2: r (abs) = 4.215e+06 (tol = 1.000e-08) r (rel) = 3.189e-03 (tol = 1.000e-08)
  Newton iteration 3: r (abs) = 8.084e+04 (tol = 1.000e-08) r (rel) = 6.116e-05 (tol = 1.000e-08)
  Newton iteration 

Now we use `PyVista` to visualize our results.

In [5]:
mesh = pv.read(os.path.join(path,'sbst','pO2000000.vtu'))
mesh.plot(jupyter_backend='pythreejs')

/home/pzuritas/.local/lib/python3.8/site-packages/pyvista/plotting/plotting.py:115: UserWarning: 
This system does not appear to be running an xserver.
PyVista will likely segfault when rendering.

Try starting a virtual frame buffer with xvfb, or using
  ``pyvista.start_xvfb()``

  warnings.warn(


AppLayout(children=(VBox(children=(HTML(value='<h3>p_O2</h3>'), Dropdown(description='Colormap:', options={'Br…